In [574]:
import re
from itertools import groupby
from datetime import datetime
from asgiref.sync import sync_to_async
import django
import os
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'oogway.settings')
django.setup()
from PostAnalyzer.models import (
    Market,
    Symbol,
    PositionSide,
    MarginMode,
    EntryTarget
)

In [575]:
def returnValue(val):
    return val.group(1) if val else None

In [576]:
def isPredictMsg(msg):
    patterns = [
        r"Symbol:\s*#?([A-Z]+)[/\s]?USDT",
        r"Take-Profit Targets:([\s\S]+?)(StopLoss|Description)",
        r"Entry Targets:([\s\S]+?)Take-Profit",
        r"Market:\s*([A-Z]+)",
        r"(StopLoss|Description):\s*([\d.]|\w+)",
    ]

    # Check if all patterns have a value
    return all(re.search(pattern, msg, re.IGNORECASE) for pattern in patterns)

In [577]:
async def findPosition(msg):
    position_match = re.search(r"Position:\s*([A-Z]+)", msg)
    
    try:
        position_value = await sync_to_async(PositionSide.objects.get)(name=returnValue(position_match).upper())
        return position_value
    except:
        return None

In [578]:
async def findMarket(msg):
    market_match = re.search(r"Market:\s*([A-Z]+)", msg, re.IGNORECASE)
    
    try:
        market_value = await sync_to_async(Market.objects.get)(name=returnValue(market_match).upper())
        return market_value
    except:
        return None


In [579]:
async def findLeverage(msg):
    leverage_match = re.search(r"Leverage:\s*(Isolated|Cross)\s*(\d+x)", msg, re.IGNORECASE)
    if leverage_match:
        leverage_type = await sync_to_async(MarginMode.objects.get)(name=returnValue(leverage_match).upper())   
        leverage_value = int(leverage_match.group(2).lower().replace("x",""))    
    else:
        leverage_type = None
        leverage_value = None
    
    return leverage_type, leverage_value

In [580]:
def findStopLoss(msg):
   return returnValue(re.search(r"StopLoss:\s*([\d.]+)", msg))

In [581]:
def findEntryTargets(msg):
    match = re.search(r"Entry Targets:([\s\S]+?)Take-Profit", msg, re.IGNORECASE)
    if match:
        extracted_data = returnValue(match)
        return [float(x.strip()) for i, x in enumerate(re.findall(r"(\d+\.\d+|\d+)", extracted_data))  if i % 2 == 1]
        

In [582]:
def findTakeProfits(msg):
    match = re.search(r"Take-Profit Targets:([\s\S]+?)(StopLoss|Description)", msg, re.IGNORECASE)
    if match:
        extracted_data = returnValue(match)
        return [float(x.strip()) for i, x in enumerate(re.findall(r"(\d+\.\d+|\d+)", extracted_data)) if i % 2 == 1]
        

In [583]:
async def findSymbol(msg):
    symbol_match = re.search(r"Symbol:\s*#?([A-Z]+)[/\s]?USDT", msg, re.IGNORECASE)
    
    try:
        symbol_value = await sync_to_async(Symbol.objects.get)(asset=returnValue(symbol_match))
        return symbol_value
    except:
        return None

In [584]:
async def findSymbol(msg):
    symbol_match = re.search(r"Symbol:\s*#?([A-Z]+)[/\s]?USDT", msg, re.IGNORECASE)
    
    try:
        symbol_value = await sync_to_async(Symbol.objects.get)(asset=returnValue(symbol_match))
        return symbol_value
    except:
        return None

In [587]:
msg = {
    "id": 3084,
    "date": "2023-12-19T12:49:50+00:00",
    "reply_to_msg_id": None,
    "message": "Symbol: #AVAX/USDT\nMarket: FUTURES\nPosition: LONG\nLeverage: Isolated 3x\n\nEntry Targets: \n1) 11.80\n2) 11.502\n\nTake-Profit Targets: \n1) 12.025\n2) 12.281\n3) 12.564\n4) 12.950\n\nStopLoss: 11.19\nB.Z",    "edit_date": None,
    "media": None,
}

msg1 = {
    "id": 3084,
    "date": "2023-12-19T12:49:50+00:00",
    "reply_to_msg_id": None,
    "message": "Symbol: #FILUSDT.P\nMarket: FUTURES\nPosition: SHORT\nLeverage: Isolated 4x\n\nEntry Targets: \n1) 5.430\n2) 5.212\n\nTake-Profit Targets: \n1) 5.517\n2) 5.608\n3) 5.715\n4) 5.823\n5) 5.982\n\nStopLoss: 5.154\nB.Z",    
    "edit_date": None,
    "media": None,
}

msg3 = {
    "id": 3084,
    "date": "2023-12-19T12:49:50+00:00",
    "reply_to_msg_id": None,
    "message": "Symbol: #BTC/USDT Market: FUTURES Position: LONG Leverage: Isolated 6x Entry Targets: 1) 30000.54 2) 32000.43 3) 33000. Take-Profit Targets: 1) 30010.66 2) 30016 3) 30019 4) 30040 StopLoss: 29010 B.Z #test",    
    "edit_date": None,
    "media": None,
}

# spot
msg2 = {
    "id": 3084,
    "date": "2023-12-19T12:49:50+00:00",
    "reply_to_msg_id": None,
    "message": "Symbol: #ETH/USDT\nMarket: SPOT\n\nEntry Targets: \n1) 2930.00\n2) 2870.00\n\nTake-Profit Targets: \n1) 3100.00\n2) 3200.00\n\n\nDescription: HOLD MODE\nمیان مدت\nB.Z",    
    "edit_date": None,
    "media": None,
}

string = msg3["message"]

# ************* FIND IS PREDICT *************
isPredict = isPredictMsg(string)
print(isPredict)


# ************* FIND SYMBOL *************
# method 1
# symbol_match = re.search(r"Symbol:(.+)", string)
# symbol_match = returnValue(symbol_match).split("USDT")[0].replace("/", "").replace(" ", "").replace("#", "")
 
# method 2
symbol = await findSymbol(string)
print(symbol)


# ************* FIND MARKET *************
market = await findMarket(string)
print(market.name)

if market.name != "SPOT":
    # ************* FIND POSITION *************
    position = await findPosition(string)
    print(position.name)

    # ************* FIND LEVERAGE *************
    marginMode, leverage = await findLeverage(string)
    print(marginMode.name, leverage)


# ************* FIND STOP LOSS *************
stopLoss = findStopLoss(string)
print(stopLoss)

# ************* FIND ENTRY TARGETS *************
entries = findEntryTargets(string)
print(entries)
 
# ************* FIND TAKE PROFITS *************
profits = findTakeProfits(string)
print(profits)

True
BTC-USDT
FUTURES
LONG
ISOLATED <class 'int'>
29010
[30000.54, 32000.43, 33000.0]
[30010.66, 30016.0, 30019.0, 30040.0]
